In [0]:
%sql
SELECT * FROM hive_metastore.etl_workbench.sufficiency_cnos

In [0]:
%sql
SELECT * FROM etl_workbench.sufficiency_pos_value

In [0]:
%sql
SELECT * FROM etl_workbench.sufficiency_pos_value_share